In [2]:
import configparser
import pandas as pd
from sqlalchemy import create_engine

cfg = configparser.ConfigParser()
cfg.read('config.ini')
db = cfg['database']

uri = (
    f"postgresql+psycopg2://{db['user']}:{db['password']}"
    f"@{db['host']}:{db['port']}/{db['dbname']}"
)
engine = create_engine(uri)


In [3]:
sql = """
SELECT
  weather_data.city_id,
  city_name,
  station_name,
  station_code,
  postal_code,
  date,
  temperature_measured_min,
  temperature_measured_max,
  temperature_measured_avg,
  temperature_predicted_min,
  temperature_predicted_max,
  temperature_predicted_avg,
  humidity_measured_min,
  humidity_measured_max,
  humidity_measured_avg,
  humidity_predicted_min,
  humidity_predicted_max,
  humidity_predicted_avg
FROM weather_data
join cities on weather_data.city_id = cities.city_id
ORDER BY date;
"""
df = pd.read_sql(sql, engine, parse_dates=['date'])


In [4]:
print(df.shape)

(6104, 18)


In [5]:
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets import interactive_output

# Dropdown options
cities = sorted(df["city_name"].unique())

# Map: label ➜ (measured_col, predicted_col)
series_map = {
    "Temperature(Minimum)":  ("temperature_measured_min",  "temperature_predicted_min"),
    "Temperature(Maximum)":  ("temperature_measured_max",  "temperature_predicted_max"),
    "Temperature(Average)":  ("temperature_measured_avg",  "temperature_predicted_avg"),
    "Relative Humidity(Minimum)":     ("humidity_measured_min",     "humidity_predicted_min"),
    "Relative Humidity(Maximum)":     ("humidity_measured_max",     "humidity_predicted_max"),
    "Relative Humidity(Average)":     ("humidity_measured_avg",     "humidity_predicted_avg"),
}

# Widgets
w_city   = widgets.Dropdown(options=cities, description="City")
w_series = widgets.Dropdown(options=list(series_map.keys()), description="Series")

# Plot function
def make_plot(city, series_label):

    # Map the selected series label to its measured and predicted column names
    col_meas, col_pred = series_map[series_label]
    
    # Filter dataframe for the chosen city and relevant columns, sort by date, drop rows with missing values
    subset = (
        df.loc[df["city_name"] == city, ["date", col_meas, col_pred]]
          .sort_values("date")
          .dropna(subset=[col_meas, col_pred])
    )
    # Initialize a Plotly figure
    fig = go.Figure()

    # Add measured data trace as a line
    fig.add_trace(go.Scatter(
        x=subset["date"], y=subset[col_meas],
        mode="lines", name="Measured",
        line=dict(color="#1f77b4")
    ))

    # Add predicted data trace as a line
    fig.add_trace(go.Scatter(
        x=subset["date"], y=subset[col_pred],
        mode="lines", name="Predicted",
        line=dict(color="#7f7f7f")
    ))
    
    # Configure layout: titles, axis labels, hover mode, template, and margins
    fig.update_layout(
        title=f"{series_label} — {city}",
        xaxis_title="Date",
        yaxis_title=series_label.split(" • ")[0],
        hovermode="x unified",
        template="plotly_white",
        legend_title=None,
        margin=dict(l=40, r=20, t=60, b=40)
    )
    # Display the plot in the notebook
    fig.show()

# Create interactive widgets linked to the make_plot function
out = interactive_output(make_plot, {"city": w_city, "series_label": w_series})
# Arrange widgets and output vertically
widgets.VBox([widgets.HBox([w_city, w_series]), out])